
# Getting Started

ML Ops is gaining a lot of popularity. This example showcases a key piece you can use to construct your automation pipeline. As we can see in the following architecture diagram, you will be deploying an AWS Step Funciton Workflow containing AWS Lambda functions that call Amazon S3, Amazon Personalize, and Amazon SNS APIs.


This package contains the source code of a Step Functions pipeline that is able to perform 
multiple actions within **Amazon Personalize**, including the following:

- Dataset Group creation
- Datasets creation and import
- Solution creation
- Solution version creation
- Campaign creation

Once the steps are completed, the step functions notifies the users of its completion through the
use of an SNS topic.

The below diagram describes the architecture of the solution:

![Architecture Diagram](../../static/imgs/ml_ops_architecture.png)

The below diagram showcases the StepFunction workflow definition:

![stepfunction definition](../../static/imgs/step_functions.png)



## Prerequisites

### Installing AWS SAM

The AWS Serverless Application Model (SAM) is an open-source framework for building serverless applications. It provides shorthand syntax to express functions, APIs, databases, and event source mappings. With just a few lines per resource, you can define the application you want and model it using YAML. During deployment, SAM transforms and expands the SAM syntax into AWS CloudFormation syntax, enabling you to build serverless applications faster.

**Install** the [AWS SAM CLI](https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/serverless-sam-cli-install.html). 
This will install the necessary tools to build, deploy, and locally test your project. In this particular example we will be using AWS SAM to build and deploy only. For additional information please visit our [documentation](https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/what-is-sam.html).


**Note:** We have pre-installed SAM CLI in this notebooks through a cloudformation life cycle policy config

Let's check what version of SAM we have installed



In [1]:
!sam --version

SAM CLI, version 1.18.2


## Directory Structure

Let's take a look at directory structure.

We have a couple artifacts that we will be using to build our MLOps pipeline.

In [17]:
!ls /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops

deploy.sh  domain  lambdas  LICENSE  shared  template.yaml


**`ml_ops/domain`**
* This directory contains the configuration file and sample data based on the domain. In this example we are going to be using the Media domain

In [18]:
!ls /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/domain

CPG  Media  Retail


**`ml_ops/lambdas`**
* This directory contains all the code that will be going into the lambda functions, these lambda functions will become a step inside the stepfunctions state machine we will deploy


In [19]:
!ls /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/lambdas

create-campaign      event-tracker	     s3lambda
create-dataset	     import-data	     wait-delete-campaign
create-datasetgroup  list-campaigns	     wait-delete-dataset
create-solution      list-datasets	     wait-delete-datasetgroup
delete-campaign      list-solutions	     wait-delete-solution
delete-dataset	     list-solution-versions  wait-delete-tracker
delete-datasetgroup  list-trackers	     wait-solution-version
delete-solution      notify
delete-tracker	     notify-delete


**`ml_ops/template.yaml`**
* This is our SAM template that will deploy the automation into our account, here we are printing just the head

In [21]:
!head /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/template.yaml

AWSTemplateFormatVersion: '2010-09-09'
Transform: AWS::Serverless-2016-10-31
Description: >

Globals:
  Function:
    Timeout: 300

Resources:



## Deploying

In order to deploy the project you will need to run the following commands:

In [25]:
!cd /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/; sam build

Building codeuri: lambdas/s3lambda/ runtime: python3.7 metadata: {} functions: ['S3Lambda']
Running PythonPipBuilder:ResolveDependencies
Running PythonPipBuilder:CopySource
Building codeuri: lambdas/notify/ runtime: python3.7 metadata: {} functions: ['Notify']
Running PythonPipBuilder:ResolveDependencies
Running PythonPipBuilder:CopySource
Building codeuri: lambdas/notify-delete/ runtime: python3.7 metadata: {} functions: ['NotifyDelete']
Running PythonPipBuilder:ResolveDependencies
Running PythonPipBuilder:CopySource
Building codeuri: lambdas/create-datasetgroup/ runtime: python3.7 metadata: {} functions: ['CreateDatasetGroup']
Running PythonPipBuilder:ResolveDependencies
Running PythonPipBuilder:CopySource
Building codeuri: lambdas/create-dataset/ runtime: python3.7 metadata: {} functions: ['CreateDataset']
Running PythonPipBuilder:ResolveDependencies
Running PythonPipBuilder:CopySource
Building codeuri: lambdas/import-data/ runtime: python3.7 metadata: {} functions: ['ImportData']
R

In [45]:
!cd /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/; sam deploy --template-file template.yaml --stack-name notebook-automation --capabilities CAPABILITY_IAM --s3-bucket $(aws cloudformation describe-stack-resources --stack-name AmazonPersonalizeImmersionDay --logical-resource-id SAMArtifactsBucket --query "StackResources[0].PhysicalResourceId" --output text)


	Deploying with following values
	Stack name                   : notebook-automation
	Region                       : us-east-1
	Confirm changeset            : False
	Deployment s3 bucket         : amazonpersonalizeimmersionday-samartifactsbucket-1v39jxpau3gqg
	Capabilities                 : ["CAPABILITY_IAM"]
	Parameter overrides          : {}
	Signing Profiles             : {}

Initiating deployment

Waiting for changeset to be created..

CloudFormation stack changeset
-------------------------------------------------------------------------------------------------
Operation                LogicalResourceId        ResourceType             Replacement            
-------------------------------------------------------------------------------------------------
+ Add                    AttachEventTrackerRole   AWS::IAM::Role           N/A                    
+ Add                    AttachEventTracker       AWS::Lambda::Function    N/A                    
+ Add                    Create

## Uploading data

Let's get the bucket that our cloudformation deployed. We will be uploading our data to this bucket, plus the configuration file to trigger the automation

In [1]:
bucket = !aws cloudformation describe-stacks --stack-name notebook-automation --query "Stacks[0].Outputs[?OutputKey=='InputBucketName'].OutputValue" --output text
bucket_name = bucket[0]
print(bucket_name)

notebook-automation-inputbucket-afxmbuzu2h16


Now that we have the bucket name, lets copy over our Media data so we can explore and upload to S3

In [2]:
!cp -R /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/domain/Media ./example

In [3]:
# Import Dependencies

import boto3
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import requests
import csv
import sys
import botocore
import uuid
from collections import defaultdict
import random
import numpy as np

from packaging import version
from botocore.exceptions import ClientError
from pathlib import Path

%matplotlib inline

# Setup Clients

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')

# We will upload our training data in these files:
raw_items_filename = "example/data/Items/items.csv"                # Do Not Change
raw_users_filename = "example/data/Users/users.csv"                # Do Not Change
raw_interactions_filename = "example/data/Interactions/interactions.csv"  # Do Not Change
items_filename = "items.csv"                # Do Not Change
users_filename = "users.csv"                # Do Not Change
interactions_filename = "interactions.csv"  # Do Not Change


In [4]:
interactions_df = pd.read_csv(raw_interactions_filename)
interactions_df.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
0,429,222,828124615,watch
1,429,227,828124615,click
2,429,595,828124615,watch
3,429,592,828124615,watch
4,429,590,828124615,watch


There are 2 ways of uploading your datasets to S3:
1. Using the boto3 SDK
1. Using the CLI

In this example we are going to use the CLI command

In [5]:
!aws s3 sync ./example/data s3://$bucket_name

upload: example/data/Items/item-meta.csv to s3://notebook-automation-inputbucket-afxmbuzu2h16/Items/item-meta.csv
upload: example/data/Interactions/interactions.csv to s3://notebook-automation-inputbucket-afxmbuzu2h16/Interactions/interactions.csv


## Starting the State Machine Execution

In order to execute the MLOps pipeline we need to provide a parameters file that will tell our state machine which names and configurations we want in our Amazon Personalize deployment.

We have prepared a parameters.json file, let's explore it

In [6]:
with open('example/params.json') as f:
  data = json.load(f)

print(json.dumps(data, indent=4, sort_keys=True))

{
    "campaigns": {
        "personalizedRankingCampaign": {
            "minProvisionedTPS": 1,
            "name": "personalizedRankingCampaign"
        },
        "simsCampaign": {
            "minProvisionedTPS": 1,
            "name": "simsCampaign"
        },
        "userPersonalizationCampaign": {
            "minProvisionedTPS": 1,
            "name": "userPersonalizationCampaign"
        }
    },
    "datasetGroup": {
        "name": "AmazonPersonalizeImmersionDay"
    },
    "datasets": {
        "Interactions": {
            "name": "InteractionsDataset",
            "schema": {
                "fields": [
                    {
                        "name": "USER_ID",
                        "type": "string"
                    },
                    {
                        "name": "ITEM_ID",
                        "type": "string"
                    },
                    {
                        "name": "EVENT_TYPE",
                        "type": "string"
      

This parameters file is set up to run at the beginning of this workshop. So let's modify a couple fields to make sure we are not overwritting our previous deployment 

In [8]:
# Dataset Groups
data['datasetGroup']['name'] = 'notebook-automation'
 
# Datasets 
data['datasets']['Interactions']['name'] = 'na-interactions-ds'
data['datasets']['Items']['name'] = 'na-items-ds'

# Solutions

data['solutions']['personalizedRanking']['name'] = 'na-personalizedRankingCampaign'
data['solutions']['sims']['name'] = 'na-simsCampaign'
data['solutions']['userPersonalization']['name'] = 'na-userPersonalizationCampaign'

# Campaigns

data['campaigns']['personalizedRankingCampaign']['name'] = 'na-personalizedRankingCampaign'
data['campaigns']['simsCampaign']['name'] = 'na-simsCampaign'
data['campaigns']['userPersonalizationCampaign']['name'] = 'na-userPersonalizationCampaign'

# Event Tracker

data['eventTracker']['name'] = 'na-eventTracker'

print(json.dumps(data, indent=4, sort_keys=True))

{
    "campaigns": {
        "personalizedRankingCampaign": {
            "minProvisionedTPS": 1,
            "name": "na-personalizedRankingCampaign"
        },
        "simsCampaign": {
            "minProvisionedTPS": 1,
            "name": "na-simsCampaign"
        },
        "userPersonalizationCampaign": {
            "minProvisionedTPS": 1,
            "name": "na-userPersonalizationCampaign"
        }
    },
    "datasetGroup": {
        "name": "notebook-automation"
    },
    "datasets": {
        "Interactions": {
            "name": "na-interactions-ds",
            "schema": {
                "fields": [
                    {
                        "name": "USER_ID",
                        "type": "string"
                    },
                    {
                        "name": "ITEM_ID",
                        "type": "string"
                    },
                    {
                        "name": "EVENT_TYPE",
                        "type": "string"
        

## Updating and uploading your parameters file to S3

First let's write the file locally

In [9]:
with open('example/params.json', 'w') as outfile:
    json.dump(data, outfile)

Now we can upload this file to S3, we are going to be using the CLI to do so

In [122]:
!aws s3 cp ./example/params.json s3://$bucket_name

upload: example/params.json to s3://notebook-automation-inputbucket-afxmbuzu2h16/params.json


## Validating the deployment

So far we have deployed the automation required lets take a look at the stepfunctions execution.

In [143]:
client = boto3.client('stepfunctions')
stateMachineArn = !aws cloudformation describe-stacks --stack-name notebook-automation --query "Stacks[0].Outputs[?OutputKey=='DeployStateMachineArn'].OutputValue" --output text
stateMachineArn= stateMachineArn[0]

In [ ]:
describe_response = client.describe_state_machine(
    stateMachineArn=stateMachineArn
)
print(json.dumps(describe_response, indent=4, sort_keys=True, default=str))

In [145]:
executions_response = client.list_executions(
    stateMachineArn=stateMachineArn,
    statusFilter='RUNNING',
    maxResults=2
)
print(json.dumps(executions_response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "345",
            "content-type": "application/x-amz-json-1.0",
            "x-amzn-requestid": "79e283e2-9f15-47aa-924e-8f7cf3ebe70c"
        },
        "HTTPStatusCode": 200,
        "RequestId": "79e283e2-9f15-47aa-924e-8f7cf3ebe70c",
        "RetryAttempts": 0
    },
    "executions": [
        {
            "executionArn": "arn:aws:states:us-east-1:144386903708:execution:DeployStateMachine-ziwRPITE9azo:2021_03_05_19_53_36",
            "name": "2021_03_05_19_53_36",
            "startDate": "2021-03-05 19:53:37.123000+00:00",
            "stateMachineArn": "arn:aws:states:us-east-1:144386903708:stateMachine:DeployStateMachine-ziwRPITE9azo",
            "status": "SUCCEEDED",
            "stopDate": "2021-03-05 21:32:27.027000+00:00"
        }
    ]
}


In [ ]:
while (True):
    if (executions_response['executions'][0]['status'] == 'SUCCEEDED'):
        break
    else:
        print (json.dumps(executions_response['executions'][0], indent=4, sort_keys=True, default=str))
        time.sleep(60)
        

{
    "executionArn": "arn:aws:states:us-east-1:674465894274:execution:DeployStateMachine-DOVbLib1H3Qs:2021_03_14_19_03_08",
    "name": "2021_03_14_19_03_08",
    "startDate": "2021-03-14 19:03:09.703000+00:00",
    "stateMachineArn": "arn:aws:states:us-east-1:674465894274:stateMachine:DeployStateMachine-DOVbLib1H3Qs",
    "status": "RUNNING"
}


### Let's look at the succeeded execution

Once your step functions are done executing, you can list the executions and describe them

In [164]:
executions_response = client.list_executions(
    stateMachineArn=stateMachineArn,
    statusFilter='SUCCEEDED',
    maxResults=2
)
print(json.dumps(executions_response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "345",
            "content-type": "application/x-amz-json-1.0",
            "x-amzn-requestid": "8a8a77c9-9378-4783-a310-a5509997bcff"
        },
        "HTTPStatusCode": 200,
        "RequestId": "8a8a77c9-9378-4783-a310-a5509997bcff",
        "RetryAttempts": 0
    },
    "executions": [
        {
            "executionArn": "arn:aws:states:us-east-1:144386903708:execution:DeployStateMachine-ziwRPITE9azo:2021_03_05_19_53_36",
            "name": "2021_03_05_19_53_36",
            "startDate": "2021-03-05 19:53:37.123000+00:00",
            "stateMachineArn": "arn:aws:states:us-east-1:144386903708:stateMachine:DeployStateMachine-ziwRPITE9azo",
            "status": "SUCCEEDED",
            "stopDate": "2021-03-05 21:32:27.027000+00:00"
        }
    ]
}


In [165]:
describe_executions_response = client.describe_execution(
    executionArn=executions_response['executions'][0]['executionArn']
)
print(json.dumps(describe_executions_response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "2648",
            "content-type": "application/x-amz-json-1.0",
            "x-amzn-requestid": "65b34b45-89cb-4369-9419-f30b91be647b"
        },
        "HTTPStatusCode": 200,
        "RequestId": "65b34b45-89cb-4369-9419-f30b91be647b",
        "RetryAttempts": 0
    },
    "executionArn": "arn:aws:states:us-east-1:144386903708:execution:DeployStateMachine-ziwRPITE9azo:2021_03_05_19_53_36",
    "input": "{\"bucket\": \"notebook-automation-inputbucket-afxmbuzu2h16\", \"currentDate\": \"2021_03_05_19_53_36\", \"params\": {\"datasetGroup\": {\"name\": \"notebook-automation\"}, \"datasets\": {\"Interactions\": {\"name\": \"na-interactions-ds\", \"schema\": {\"type\": \"record\", \"name\": \"Interactions\", \"namespace\": \"com.amazonaws.personalize.schema\", \"fields\": [{\"name\": \"USER_ID\", \"type\": \"string\"}, {\"name\": \"ITEM_ID\", \"type\": \"string\"}, {\"name\": \"EVENT_TYPE\", \"type\": \"str

### Let's look at the input that was delivered to the State Machine

As we can see below, this is the input from our Parameters file we uploaded to S3. This input json was then passed to lambda functions in the state machine to utilize across Amazon Personalize APIs

In [166]:
print(json.dumps(json.loads(describe_executions_response['input']), indent=4, sort_keys=True, default=str))

{
    "bucket": "notebook-automation-inputbucket-afxmbuzu2h16",
    "currentDate": "2021_03_05_19_53_36",
    "params": {
        "campaigns": {
            "personalizedRankingCampaign": {
                "minProvisionedTPS": 1,
                "name": "na-personalizedRankingCampaign"
            },
            "simsCampaign": {
                "minProvisionedTPS": 1,
                "name": "na-simsCampaign"
            },
            "userPersonalizationCampaign": {
                "minProvisionedTPS": 1,
                "name": "na-userPersonalizationCampaign"
            }
        },
        "datasetGroup": {
            "name": "notebook-automation"
        },
        "datasets": {
            "Interactions": {
                "name": "na-interactions-ds",
                "schema": {
                    "fields": [
                        {
                            "name": "USER_ID",
                            "type": "string"
                        },
                      

### Let's look at the time stamps

As we can see below, this is the input from our Parameters file we uploaded to S3. This input json was then passed to lambda functions in the state machine to utilize across Amazon Personalize APIs

In [167]:
print("Start Date:")
print(json.dumps(describe_executions_response['startDate'], indent=4, sort_keys=True, default=str))
print("Stop Date:")
print(json.dumps(describe_executions_response['stopDate'], indent=4, sort_keys=True, default=str))
print("Elapsed Time: ")
elapsed_time = describe_executions_response['stopDate'] - describe_executions_response['startDate']
print(elapsed_time)

Start Date:
"2021-03-05 19:53:37.123000+00:00"
Stop Date:
"2021-03-05 21:32:27.027000+00:00"
Elapsed Time: 
1:38:49.904000


As we see above, the automation around an hour with fourty minutes.

If you are interested in deploying this example in your environment, visit our [Github Samples Page](https://github.com/aws-samples/amazon-personalize-samples/tree/master/next_steps/operations/ml_ops) to download the latest codebase.